In [1]:
from openbabel import pybel
from MZMol import MZMol
import numpy as np
import torch
from DirectPIP import evp, evdpdmor
from msa import basis, gradient
from pes_shell import pes_shell
import json

| Item            | Test |
| :-------------: | :--: |
| distance vector | pass |
| morse like      | pass |
| pip             | pass |
| dr/dxyz          | pass |
| dp/dxyz          | pass |

# Load Molecule

In [2]:
_mol = next(pybel.readfile(format="xyz", filename="test.xyz"))
mol = MZMol(_mol)

# Distance Vector $\boldsymbol{r}$ & Morse-Like $\boldsymbol{m}$

In [3]:
def morse(r: np.array, alpha=1.0) -> np.array:
    return np.exp(-r / alpha)

In [4]:
%%timeit

r = mol.distance_vector
mor = morse(r)

7.09 µs ± 511 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [5]:
r = mol.distance_vector
mor = morse(r)
print(f"r = {r}")
print(f"m = {mor}")

r = [1.19804053 0.99253339 1.05928332 1.21859636 1.81793964 1.80035646
 1.95859103 1.66341587 1.88760548 1.81186328]
m = [0.30178497 0.37063653 0.3467042  0.29564485 0.16235993 0.16523998
 0.14105703 0.1894906  0.15143399 0.16334949]


In [6]:
%%timeit

msa_mor = pes_shell.evx(mol.coords.T)

1 µs ± 23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
msa_mor = pes_shell.evx(mol.coords.T)
print(f"msa_m = {msa_mor}")

msa_m = [0.301785   0.37063652 0.34670419 0.29564482 0.16235992 0.16523999
 0.14105703 0.18949059 0.15143397 0.16334948]


In [8]:
(msa_mor - mor < 1e-6).all()

True

# PIP

$$
\boldsymbol{p} = \hat{P}(\boldsymbol{m})
$$

where $\boldsymbol{m}$ is the Morse-Like vector and $\hat{P}$ is the permutation invariant polynomial operator.

In [9]:
# load PIP basis
with open("MOL_1_4_5.json") as f:
    basis_list = json.load(f)

In [10]:
%%timeit

direct_poly = np.array([evp(torch.from_numpy(mor), torch.from_numpy(np.array(b))).clone().detach() for b in basis_list])

9.33 ms ± 686 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
direct_poly = np.array([evp(torch.from_numpy(mor), torch.from_numpy(np.array(b))).clone().detach() for b in basis_list])
np.savetxt("direct_poly.txt", direct_poly)
print(f"p[:20] = {direct_poly[:20]}")

p[:20] = [1.         0.972931   1.31477056 0.07827334 0.31548199 0.63682436
 0.64235668 0.64628316 0.15908408 0.43605529 0.05075697 0.01704662
 0.01704728 0.1028042  0.10291148 0.20740288 0.10457936 0.10389676
 0.41917048 0.10572168]


In [12]:
%%timeit

msa_mono = basis.evmono(mor)
msa_poly = basis.evpoly(msa_mono)

1.44 µs ± 39.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
%%timeit

msa_poly = basis.bemsav(mor)

898 ns ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [14]:
# msa_poly = basis.bemsav(mor)
msa_mono = basis.evmono(mor)
msa_poly = basis.evpoly(msa_mono)
np.savetxt("msa_poly.txt", msa_poly)
print(f"p[:20] = {msa_poly[:20]}")

p[:20] = [1.         0.972931   1.31477056 0.07827334 0.31548199 0.63682436
 0.64235668 0.64628316 0.15908408 0.43605529 0.05075697 0.01704662
 0.01704728 0.1028042  0.10291148 0.20740288 0.10457936 0.10389676
 0.41917048 0.10572168]


In [15]:
(msa_poly - direct_poly < 1e-6).all()

True

# Jacobian Matrix of Distance Vector $\mathbf{r}$

$$
J_{r(x, y, z)} = \begin{bmatrix}
\frac{\partial r_1}{\partial x_1} & \frac{\partial r_1}{\partial y_1} & \frac{\partial r_1}{\partial z_1} & \cdots & \frac{\partial r_1}{\partial x_n} & \frac{\partial r_1}{\partial y_n} & \frac{\partial r_1}{\partial z_n} \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
\frac{\partial r_k}{\partial x_1} & \frac{\partial r_k}{\partial y_1} & \frac{\partial r_k}{\partial z_1} & \cdots & \frac{\partial r_k}{\partial x_n} & \frac{\partial r_k}{\partial y_n} & \frac{\partial r_k}{\partial z_n} \\
\end{bmatrix}
$$

In [16]:
%%timeit

J_r_xyz = mol.J_r_xyz

149 µs ± 6.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
J_r_xyz = mol.J_r_xyz
np.savetxt("J_r_xyz.txt", J_r_xyz)
print(f"J_r_xyz[0, :] = {J_r_xyz[0, :]}")

J_r_xyz[0, :] = [-0.74696972  0.65982743 -0.0816333   0.74696972 -0.65982743  0.0816333
  0.          0.          0.          0.          0.          0.
  0.          0.          0.        ]


In [18]:
%%timeit

msa_drdx = pes_shell.evdrdx(mol.coords.T)

1.3 µs ± 34.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [19]:
msa_drdx = pes_shell.evdrdx(mol.coords.T)
np.savetxt("msa_drdx.txt", msa_drdx.T)
print(f"msa_drdx[:, 0] = {msa_drdx[:, 0]}")

msa_drdx[:, 0] = [-0.7469697   0.6598274  -0.08163331  0.7469697  -0.6598274   0.08163331
  0.          0.          0.          0.          0.          0.
  0.          0.          0.        ]


In [20]:
(msa_drdx - J_r_xyz.T < 1e-6).all()

True

# Direct PIP Gradient

$$
J_{\boldsymbol{p}(x, y, z)} = J_{\boldsymbol{p}(\boldsymbol{m})} J_{\boldsymbol{m}(\boldsymbol{r})} J_{\boldsymbol{r}(x, y, z)}
$$

In [21]:
def ev_J_mor_r(r: np.array, alpha=1.0) -> np.array:
    return -np.diag(morse(r, alpha=alpha)) / alpha

In [22]:
J_mor_r = ev_J_mor_r(r)

In [23]:
%%timeit

J_P_mor = np.array([evdpdmor(torch.from_numpy(mor), torch.from_numpy(np.array(b))).detach().numpy() for b in basis_list])

33.5 ms ± 566 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
J_P_mor = np.array([evdpdmor(torch.from_numpy(mor), torch.from_numpy(np.array(b))).detach().numpy() for b in basis_list])

In [25]:
direct_J_P_xyz_T = (J_P_mor @ J_mor_r @ J_r_xyz).T
np.savetxt("direct_J_P_xyz_T.txt", direct_J_P_xyz_T)
print("direct_J_P_xyz_T[:, 1] = ", direct_J_P_xyz_T[:, 1]) # all elements in 1st column is 0

direct_J_P_xyz_T[:, 1] =  [ 0.          0.          0.         -0.27688001  0.27195805 -0.04485967
  0.17643197 -0.0694497  -0.35723558 -0.160164   -0.3407535   0.15548375
  0.26061203  0.13824515  0.24661149]


# MSA PIP Gradient

In [26]:
natom = mol.natom
print(f"{natom = }")

natom = 5


In [27]:
%%timeit

msa_J_P_xyz_T = np.array([gradient.dbemsav(
    msa_drdx,
    msa_mono,
    msa_poly,
    j
) for j in range(1, 3 * natom + 1)])

36.6 µs ± 631 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
msa_J_P_xyz_T = np.array([gradient.dbemsav(
    msa_drdx,
    msa_mono,
    msa_poly,
    j
) for j in range(1, 3 * natom + 1)])

np.savetxt("msa_J_P_xyz_T.txt", msa_J_P_xyz_T)
print("msa_J_P_xyz_T[:, 1] = ", msa_J_P_xyz_T[:, 1])

msa_J_P_xyz_T[:, 1] =  [-0.         -0.         -0.         -0.27688001  0.27195805 -0.04485967
  0.17643197 -0.06944971 -0.35723558 -0.160164   -0.34075348  0.15548375
  0.26061203  0.13824514  0.2466115 ]


In [29]:
(msa_J_P_xyz_T - direct_J_P_xyz_T < 1e-3).all()

True